In [1]:
import pandas as pd
import numpy as np
import json
from glob import glob
import os

In [3]:
DATAROOT = '../data'

RAWPATH = os.path.join(DATAROOT,'raw/')

OUTPUTPATH = os.path.join(DATAROOT, 'processed/')
!mkdir $OUTPUTPATH

In [4]:
WATER_STATION_FP = os.path.join(RAWPATH ,'ca_rs_locations_for_sensors.csv')
WATER_STREAMFLOW_FP =  os.path.join(RAWPATH, 'Streamflow_data_USGS_original/Unfiltered/*.csv')

In [5]:
streamflow = pd.concat([pd.read_csv(i, 
                            index_col=0,
                            parse_dates=['Date'],
                            usecols=['site_no', 'Date', 'X_00060_00003']
                           ) for i in \
                glob(WATER_STREAMFLOW_FP)]).reset_index()
streamflow.columns = ['site_id', 'date', 'streamflow']

streamflow = streamflow.astype(
        {
            'site_id': str,
            'streamflow': float
        }
)

/Users/syeehyn/opt/anaconda3/envs/scipy-ml/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
streamflow.head()

,site_id,date,streamflow
0,10250800,1962-10-01,0.3
1,10250800,1962-10-02,0.3
2,10250800,1962-10-03,0.3
3,10250800,1962-10-04,0.3
4,10250800,1962-10-05,0.3


In [7]:
station_reference = pd.read_csv(WATER_STATION_FP, 
                                usecols=['STAID', 'STANAME', 'HUC02', 'LAT_GAGE', \
                                         'LNG_GAGE', 'HCDN-2009', 'CLASS', 'AGGECOREGION'],
                                index_col=None)
station_reference.columns = ['site_id', 'site_name', 'huc02', 'latitude', 'longitude', \
                             'hucdn_2009', 'is_ref', 'aggecoregion']
#a hacky patch that cast id into int
station_reference.site_id = station_reference.site_id.astype(int)
# cast binary into boolean type
station_reference.hucdn_2009 = (station_reference.hucdn_2009 == 'yes')
station_reference.is_ref = (station_reference.is_ref == 'Ref')
station_reference = station_reference.astype(
            {
                'site_id': str,
                'site_name': str,
                'huc02': 'category',
                'latitude': float,
                'longitude': float,
                'hucdn_2009': bool,
                'aggecoregion': 'category'
            }
)
#lowercase strings

station_reference.site_name = station_reference.site_name.str.lower()
station_reference.aggecoregion = station_reference.aggecoregion.str.lower()

In [8]:
station_reference.head()

,site_id,site_name,huc02,latitude,longitude,hucdn_2009,is_ref,aggecoregion
0,9423350,caruthers c nr ivanpah ca,15,35.244989,-115.298876,True,True,westxeric
1,10250800,darwin c nr darwin ca,18,36.320496,-117.523955,False,False,westxeric
2,10251300,"amargosa river at tecopa, ca",18,35.848577,-116.230024,False,False,westxeric
3,10255700,san felipe c nr julian ca,18,33.118654,-116.435293,False,False,westxeric
4,10255800,coyote c nr borrego springs ca,18,33.373645,-116.427514,False,True,westxeric


In [9]:
# all streamflow data is in the site refence data
assert all(np.isin(streamflow.site_id.unique(), station_reference.site_id))

In [10]:
complete_datum = streamflow.pivot_table(index='date', columns='site_id', values='streamflow')

In [11]:
start_date, end_date = complete_datum.index[[0, -1]]

In [12]:
start_date

Timestamp('1891-01-01 00:00:00')

In [13]:
end_date

Timestamp('2019-10-14 00:00:00')

In [14]:
streamflow.to_csv(os.path.join(OUTPUTPATH, 'streamflow_parsed.csv'), index=False)
complete_datum.to_csv(os.path.join(OUTPUTPATH, 'streamflow_sequence.csv'))
station_reference.to_csv(os.path.join(OUTPUTPATH, 'site_reference.csv'), index=False)

In [15]:
metadata = {
    'streamflow_parsed': {
        'site_id': {
            'type': 'string',
            'description': 'the site id of the observation',
            'metadata': f'{streamflow.site_id.nunique()} number of unique sites'
        },
        'date': {
            'type': 'date',
            'description': 'the dates of each row',
            'startsAt': '1891-01-01',
            'endsAt': '2019-10-14'
        },
        'streamflow': {
            'type': 'float',
            'description': 'stream flow on daily average',
        }
    },
    'streamflow_sequence': {
        'description': 'daily streamflow information on each sites, columns: on each site, rows: on each day'
    },
    'site_reference': {
        'site_id': {
            'type': 'string',
            'description': 'the site id of the observation',
            'metadata': f'{station_reference.site_id.nunique()} number of unique sites'
        },
        'site_name': {
            'type': 'string',
            'description': 'the station name in lower case'
        },
        'huc02': {
            'type': 'string',
            'description': 'the huc02 region of the site',
            'metadata': list(station_reference.huc02.unique())
        },
        'latitude': {
            'type': 'float',
            'description': 'the corresponding latitude of the site in epsg 3857'
        },
        'longitude': {
            'type': 'float',
            'description': 'the corresponding longitude of the site in epsg 3857'
        },
        'hucdn_2009': {
            'type': 'boolean',
            'description': 'True if is in hucdn_2009, False otherwise'
        },
        'is_ref': {
            'type': 'boolean',
            'description': 'True if is reference site, False otherwise'
        },
        'aggecoregion':{
            'type': 'string',
            'description': 'the aggecoregion of the site',
            'metadata': list(station_reference.aggecoregion.unique())
        }
    }
}
metadata['streamflow_parsed']['streamflow']['metadata'] = streamflow.streamflow.describe().to_dict()

In [16]:
with open(os.path.join(OUTPUTPATH, 'METADATA.json'), 'w+') as f:
    json.dump(metadata, f)

In [17]:
1979

1979